In [1]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [2]:
groq_api_key=""

In [3]:
llm=ChatGroq(model="Gemma2-9b-It",groq_api_key=groq_api_key)

#### Creating a Tool from Strach

In [4]:
from langchain.agents import tool

In [5]:
@tool
def search(query:str)->str:
    """Serach for weather online"""
    return "42f"

In [6]:
search.name

'search'

In [7]:
search.description

'Serach for weather online'

In [8]:
search.args

{'query': {'title': 'Query', 'type': 'string'}}

In [9]:
from pydantic import BaseModel,Field
class SearchInput(BaseModel):
    query: str = Field(description="Thing to search for")

In [10]:
@tool(args_schema=SearchInput)
def search(query:str)->str:
    """Search for the weather online"""
    return "42f"

In [11]:
search.args

{'query': {'description': 'Thing to search for',
  'title': 'Query',
  'type': 'string'}}

In [12]:
search.run("lucknow")

'42f'

In [13]:
import requests
from pydantic import BaseModel, Field
import datetime

# Define the input schema
class OpenMeteoInput(BaseModel):
    latitude: float = Field(..., description="Latitude of the location to fetch weather data for")
    longitude: float = Field(..., description="Longitude of the location to fetch weather data for")

@tool(args_schema=OpenMeteoInput)
def get_current_temperature(latitude: float, longitude: float) -> dict:
    """Fetch current temperature for given coordinates."""
    
    BASE_URL = "https://api.open-meteo.com/v1/forecast"
    
    # Parameters for the request
    params = {
        'latitude': latitude,
        'longitude': longitude,
        'hourly': 'temperature_2m',
        'forecast_days': 1,
    }

    # Make the request
    response = requests.get(BASE_URL, params=params)
    
    if response.status_code == 200:
        results = response.json()
    else:
        raise Exception(f"API Request failed with status code: {response.status_code}")

    current_utc_time = datetime.datetime.utcnow()
    time_list = [datetime.datetime.fromisoformat(time_str.replace('Z', '+00:00')) for time_str in results['hourly']['time']]
    temperature_list = results['hourly']['temperature_2m']
    
    closest_time_index = min(range(len(time_list)), key=lambda i: abs(time_list[i] - current_utc_time))
    current_temperature = temperature_list[closest_time_index]
    
    return f'The current temperature is {current_temperature}°C'

In [14]:
get_current_temperature.name

'get_current_temperature'

In [15]:
get_current_temperature.description

'Fetch current temperature for given coordinates.'

In [16]:
get_current_temperature.args

{'latitude': {'description': 'Latitude of the location to fetch weather data for',
  'title': 'Latitude',
  'type': 'number'},
 'longitude': {'description': 'Longitude of the location to fetch weather data for',
  'title': 'Longitude',
  'type': 'number'}}

In [17]:
from langchain.tools.render import format_tool_to_openai_function

In [18]:
format_tool_to_openai_function(get_current_temperature)

a:\LANGCHAIN\langchainenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The function `format_tool_to_openai_function` was deprecated in LangChain 0.1.16 and will be removed in 0.3.0. Use langchain_core.utils.function_calling.convert_to_openai_function() instead.
  warn_deprecated(


{'name': 'get_current_temperature',
 'description': 'Fetch current temperature for given coordinates.',
 'parameters': {'properties': {'latitude': {'description': 'Latitude of the location to fetch weather data for',
    'type': 'number'},
   'longitude': {'description': 'Longitude of the location to fetch weather data for',
    'type': 'number'}},
  'required': ['latitude', 'longitude'],
  'type': 'object'}}

In [19]:
get_current_temperature({"latitude": 13, "longitude": 14})

a:\LANGCHAIN\langchainenv\lib\site-packages\langchain_core\_api\deprecation.py:139: LangChainDeprecationWarning: The method `BaseTool.__call__` was deprecated in langchain-core 0.1.47 and will be removed in 0.3.0. Use invoke instead.
  warn_deprecated(


'The current temperature is 27.6°C'

In [20]:
import wikipedia
@tool
def search_wikipedia(query: str) -> str:
    """Run Wikipedia search and get page summaries."""
    page_titles = wikipedia.search(query)
    summaries = []
    for page_title in page_titles[: 3]:
        try:
            wiki_page =  wikipedia.page(title=page_title, auto_suggest=False)
            summaries.append(f"Page: {page_title}\nSummary: {wiki_page.summary}")
        except (
            self.wiki_client.exceptions.PageError,
            self.wiki_client.exceptions.DisambiguationError,
        ):
            pass
    if not summaries:
        return "No good Wikipedia Search Result was found"
    return "\n\n".join(summaries)

In [21]:
search_wikipedia.name

'search_wikipedia'

In [22]:
search_wikipedia.description

'Run Wikipedia search and get page summaries.'

In [23]:
format_tool_to_openai_function(search_wikipedia)

{'name': 'search_wikipedia',
 'description': 'Run Wikipedia search and get page summaries.',
 'parameters': {'type': 'object',
  'properties': {'query': {'type': 'string'}},
  'required': ['query']}}

In [24]:
search_wikipedia({"query": "langchain"})

"Page: LangChain\nSummary: LangChain is a software framework that helps facilitate the integration of large language models (LLMs) into applications. As a language model integration framework, LangChain's use-cases largely overlap with those of language models in general, including document analysis and summarization, chatbots, and code analysis.\n\nPage: DataStax\nSummary: DataStax, Inc. is a real-time data for AI company based in Santa Clara, California. Its product Astra DB is a cloud database-as-a-service based on Apache Cassandra. DataStax also offers DataStax Enterprise (DSE), an on-premises database built on Apache Cassandra, and Astra Streaming, a messaging and event streaming cloud service based on Apache Pulsar. As of June 2022, the company has roughly 800 customers distributed in over 50 countries.\n\nPage: Retrieval-augmented generation\nSummary: Retrieval augmented generation (RAG) is a technique that grants generative artificial intelligence models information retrieval c

In [25]:
from langchain.chains.openai_functions.openapi import openapi_spec_to_openai_fn
from langchain.utilities.openapi import OpenAPISpec

In [26]:
text = """
{
  "openapi": "3.0.0",
  "info": {
    "version": "1.0.0",
    "title": "Swagger Petstore",
    "license": {
      "name": "MIT"
    }
  },
  "servers": [
    {
      "url": "http://petstore.swagger.io/v1"
    }
  ],
  "paths": {
    "/pets": {
      "get": {
        "summary": "List all pets",
        "operationId": "listPets",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "limit",
            "in": "query",
            "description": "How many items to return at one time (max 100)",
            "required": false,
            "schema": {
              "type": "integer",
              "maximum": 100,
              "format": "int32"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "A paged array of pets",
            "headers": {
              "x-next": {
                "description": "A link to the next page of responses",
                "schema": {
                  "type": "string"
                }
              }
            },
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pets"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      },
      "post": {
        "summary": "Create a pet",
        "operationId": "createPets",
        "tags": [
          "pets"
        ],
        "responses": {
          "201": {
            "description": "Null response"
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    },
    "/pets/{petId}": {
      "get": {
        "summary": "Info for a specific pet",
        "operationId": "showPetById",
        "tags": [
          "pets"
        ],
        "parameters": [
          {
            "name": "petId",
            "in": "path",
            "required": true,
            "description": "The id of the pet to retrieve",
            "schema": {
              "type": "string"
            }
          }
        ],
        "responses": {
          "200": {
            "description": "Expected response to a valid request",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Pet"
                }
              }
            }
          },
          "default": {
            "description": "unexpected error",
            "content": {
              "application/json": {
                "schema": {
                  "$ref": "#/components/schemas/Error"
                }
              }
            }
          }
        }
      }
    }
  },
  "components": {
    "schemas": {
      "Pet": {
        "type": "object",
        "required": [
          "id",
          "name"
        ],
        "properties": {
          "id": {
            "type": "integer",
            "format": "int64"
          },
          "name": {
            "type": "string"
          },
          "tag": {
            "type": "string"
          }
        }
      },
      "Pets": {
        "type": "array",
        "maxItems": 100,
        "items": {
          "$ref": "#/components/schemas/Pet"
        }
      },
      "Error": {
        "type": "object",
        "required": [
          "code",
          "message"
        ],
        "properties": {
          "code": {
            "type": "integer",
            "format": "int32"
          },
          "message": {
            "type": "string"
          }
        }
      }
    }
  }
}
"""

In [27]:
specs=OpenAPISpec.from_text(text)

Attempting to load an OpenAPI 3.0.0 spec.  This may result in degraded performance. Convert your OpenAPI spec to 3.1.* spec for better support.


AttributeError: 'super' object has no attribute 'parse_obj'

In [28]:
pet_openai_functions,pet_callables=openapi_spec_to_openai_fn(specs)

NameError: name 'specs' is not defined

In [29]:
llm = ChatGroq(temperature=0).bind(functions=pet_openai_functions)

NameError: name 'pet_openai_functions' is not defined

In [30]:
llm.invoke("what are three pets names")

AIMessage(content="Here are three pet names:\n\n1. Luna\n2. Max\n3. Bella \n\n\nLet me know if you'd like more ideas!  🐶🐱🐹\n", response_metadata={'token_usage': {'completion_tokens': 38, 'prompt_tokens': 14, 'total_tokens': 52, 'completion_time': 0.069090909, 'prompt_time': 7.745e-05, 'queue_time': 0.012946639000000001, 'total_time': 0.069168359}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run-8f3db57e-b2ab-4c59-8818-0ebd076e8e1a-0', usage_metadata={'input_tokens': 14, 'output_tokens': 38, 'total_tokens': 52})

In [31]:
model.invoke("tell me about pet with id 42")

NameError: name 'model' is not defined

## Routing¶

In lesson 3, we show an example of function calling deciding between two candidate functions.

Given our tools above, let's format these as OpenAI functions and show this same behavior.

In [32]:
functions=[format_tool_to_openai_function(f) for f in [search_wikipedia,get_current_temperature]]
model=llm.bind(functions=functions)

In [33]:
model.invoke("what is weather in lucknow right now?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_x9zv', 'function': {'arguments': '{"latitude":26.8444,"longitude":80.9462}', 'name': 'get_current_temperature'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 102, 'prompt_tokens': 1108, 'total_tokens': 1210, 'completion_time': 0.185454545, 'prompt_time': 0.045779486, 'queue_time': 0.008361524000000002, 'total_time': 0.231234031}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f2608a0e-b878-491f-b8d2-3b228301425b-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 26.8444, 'longitude': 80.9462}, 'id': 'call_x9zv', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1108, 'output_tokens': 102, 'total_tokens': 1210})

In [34]:
model.invoke("what is langchain?")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_5a29', 'function': {'arguments': '{"query":"LangChain"}', 'name': 'search_wikipedia'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 82, 'prompt_tokens': 1104, 'total_tokens': 1186, 'completion_time': 0.149090909, 'prompt_time': 0.037607559, 'queue_time': 0.002696081000000003, 'total_time': 0.186698468}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-2a15efcd-c7e8-40c8-855a-5200ee6e1e50-0', tool_calls=[{'name': 'search_wikipedia', 'args': {'query': 'LangChain'}, 'id': 'call_5a29', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1104, 'output_tokens': 82, 'total_tokens': 1186})

In [35]:
from langchain.prompts import ChatPromptTemplate
prompt=ChatPromptTemplate(
    [
        ("system","You are helpful but saassy assistant"),
        ("user","{input}"),
    ]
)

chain=prompt|model

In [36]:
chain.invoke({"input":"what is weather in lucknow right now?"})

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_2h7c', 'function': {'arguments': '{"latitude":26.8448,"longitude":80.9481}', 'name': 'get_current_temperature'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 103, 'prompt_tokens': 1118, 'total_tokens': 1221, 'completion_time': 0.187272727, 'prompt_time': 0.04700923, 'queue_time': 0.0030506699999999984, 'total_time': 0.234281957}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-f3dc6213-a2e6-4874-afd8-466657067e05-0', tool_calls=[{'name': 'get_current_temperature', 'args': {'latitude': 26.8448, 'longitude': 80.9481}, 'id': 'call_2h7c', 'type': 'tool_call'}], usage_metadata={'input_tokens': 1118, 'output_tokens': 103, 'total_tokens': 1221})

In [37]:
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser

In [38]:
chain=prompt|model|OpenAIFunctionsAgentOutputParser()

In [39]:
result = chain.invoke({"input": "what is the weather in sf right now"})

In [40]:
result

AgentFinish(return_values={'output': ''}, log='')

In [41]:
type(result)

langchain_core.agents.AgentFinish

In [42]:
result.tool

AttributeError: 'AgentFinish' object has no attribute 'tool'

In [43]:
result.tool_input

AttributeError: 'AgentFinish' object has no attribute 'tool_input'

In [44]:
get_current_temperature(result.tool_input)

AttributeError: 'AgentFinish' object has no attribute 'tool_input'

In [45]:
result = chain.invoke({"input": "hi!"})

In [46]:
type(result)

langchain_core.agents.AgentFinish

In [47]:
result.return_values

{'output': ''}

In [48]:
from langchain.schema.agent import AgentFinish
def route(result):
    if isinstance(result, AgentFinish):
        return result.return_values['output']
    else:
        tools = {
            "search_wikipedia": search_wikipedia, 
            "get_current_temperature": get_current_temperature,
        }
        return tools[result.tool].run(result.tool_input)

In [49]:
chain = prompt | model | OpenAIFunctionsAgentOutputParser() | route

In [50]:
result = chain.invoke({"input": "What is the weather in san francisco right now?"})

In [51]:
result

''

In [52]:
result = chain.invoke({"input": "What is langchain?"})

In [53]:
result

''